# Diabetes Health Indicators - Data Analyse

**Dataset:** CDC BRFSS 2015 (Behavioral Risk Factor Surveillance System)

Dit notebook:
1. Kloont de repository en installeert dependencies
2. Downloadt de dataset van Kaggle
3. Voert data cleaning uit via `data_cleaner.py`
4. Splitst de data op per onderzoeksvraag
5. Toont de resultaten per vraag

## Stap 1: Repository klonen en dependencies installeren

In [ ]:
!git clone https://github.com/Pablo-Atanes/Minor-bigdata.git
%cd Minor-bigdata
!pip install -q -r requirements.txt

## Stap 2: Dataset downloaden van Kaggle

De `data/` map staat in `.gitignore` en zit dus niet in de repository.
We downloaden de dataset via `kagglehub`.

**Let op:** Bij de eerste keer wordt gevraagd om je Kaggle credentials.
Deze kun je aanmaken op https://www.kaggle.com/settings > API > Create New Token.

In [ ]:
!pip install -q kagglehub

import kagglehub
import shutil
import os

# Dataset downloaden van Kaggle
path = kagglehub.dataset_download("alexteboul/diabetes-health-indicators-dataset")
print(f"Dataset gedownload naar: {path}")

# CSV bestanden kopieren naar data/ map
os.makedirs("data", exist_ok=True)
for f in os.listdir(path):
    if f.endswith(".csv"):
        shutil.copy(os.path.join(path, f), os.path.join("data", f))
        print(f"Gekopieerd: {f}")

print("\nBestanden in data/:")
!ls data/

## Stap 3: Onderzoeksvragen bekijken

We tonen de inhoud van `research_questions.md` om te bevestigen welke variabelen we nodig hebben.

In [ ]:
!cat research_questions.md

## Stap 4: Data laden en opschonen

We gebruiken de `data_cleaner.py` module uit de repository.
Deze voert de volgende stappen uit:
1. Dubbele rijen verwijderen
2. Rijen met ontbrekende waarden verwijderen
3. Datatypen converteren naar integers
4. Waardebereiken valideren (binaire kolommen 0/1, ordinale kolommen binnen bereik)
5. BMI uitschieters filteren (12-98)

In [ ]:
from data_procesing.data_cleaner import load_data, clean_data, get_data_for_questions, save_questions_to_json

# Data laden
df = load_data()
print(f"Ruwe data: {df.shape[0]} rijen, {df.shape[1]} kolommen")
print(f"Kolommen: {df.columns.tolist()}\n")

# Data opschonen
df = clean_data(df)

print(f"\nEerste 5 rijen van de opgeschoonde data:")
df.head()

In [ ]:
print("Datatypen:")
print(df.dtypes)
print(f"\nVorm: {df.shape}")

## Stap 5: Data opsplitsen per onderzoeksvraag

| Vraag | Onderwerp | Kolommen |
|-------|-----------|----------|
| Q1 | Fruit Consumptie & Diabetes | `Diabetes_binary`, `Fruits` |
| Q2 | Lifestyle Patronen (Apriori) | `Diabetes_binary`, `Smoker`, `PhysActivity`, `Fruits`, `Veggies`, `HvyAlcoholConsump` |
| Q3 | Socio-economische Status | `Diabetes_binary`, `Income`, `Education` |
| Q4 | Cumulatief Lifestyle Effect | `Diabetes_binary`, `Smoker`, `PhysActivity`, `Fruits`, `HvyAlcoholConsump`, `Unhealthy_Lifestyle_Score` |
| Q5 | Cholesterol + Bloeddruk | `Diabetes_binary`, `HighBP`, `HighChol`, `HighBP_x_HighChol` |

In [ ]:
# Datasets per vraag genereren en opslaan als JSON
questions = get_data_for_questions(df)
save_questions_to_json(questions)

## Stap 6: Data inspectie per onderzoeksvraag

Hieronder tonen we voor elke onderzoeksvraag de eerste rijen en beschrijvende statistieken.

### Onderzoeksvraag 1: Fruit Consumptie & Diabetes
> "Heeft het regelmatig eten van fruit (>=1x per dag) invloed op de kans op diabetes?"

In [ ]:
q1 = questions["q1_fruit_diabetes"]
print(f"Vorm: {q1.shape}")
display(q1.head())
display(q1.describe())

### Onderzoeksvraag 2: Lifestyle Patronen & Diabetes (Associatieregels)
> "Welke lifestyle patronen komen vaak samen voor bij mensen met diabetes?"

In [ ]:
q2 = questions["q2_lifestyle_patterns"]
print(f"Vorm: {q2.shape}")
display(q2.head())
display(q2.describe())

### Onderzoeksvraag 3: Socio-economische Status & Diabetes
> "Heeft een lagere socio-economische status een negatief effect op de kans op diabetes?"

In [ ]:
q3 = questions["q3_socioeconomic"]
print(f"Vorm: {q3.shape}")
display(q3.head())
display(q3.describe())

### Onderzoeksvraag 4: Cumulatief Effect Lifestyle Factors
> "Hoe verandert het diabetes risico wanneer meerdere ongezonde lifestyle factors gecombineerd worden?"

De afgeleide `Unhealthy_Lifestyle_Score` wordt berekend als:
```
Smoker + (1 - PhysActivity) + (1 - Fruits) + HvyAlcoholConsump
```
Bereik: 0 (geen risicofactoren) tot 4 (alle risicofactoren)

In [ ]:
q4 = questions["q4_cumulative_lifestyle"]
print(f"Vorm: {q4.shape}")
display(q4.head())
display(q4.describe())

print("\nVerdeling Unhealthy_Lifestyle_Score:")
print(q4["Unhealthy_Lifestyle_Score"].value_counts().sort_index())

### Onderzoeksvraag 5: Cholesterol + Bloeddruk & Diabetes
> "Verhoogt de combinatie van hoog cholesterol en hoge bloeddruk het diabetes risico synergistisch?"

De interactieterm `HighBP_x_HighChol` is 1 wanneer beide condities aanwezig zijn.

In [ ]:
q5 = questions["q5_cholesterol_bloodpressure"]
print(f"Vorm: {q5.shape}")
display(q5.head())
display(q5.describe())